
<h1><center> The GeMSE Analysis Infrastructure </center></h1>

<br>
<br>
<br>
<br>
<br>

This jupyter notebook is meant to provide (backup) documentation of the GeMSE analysis infrastruture and to allow easy access to it.

<br>

Chapters 1 and 2 feature documentation of the explicit use of the analysis infrastructure - both remotely on the `bcfr102` machine and also on your local hardware by running a `Docker` container. This is probably what you are looking for when you were referred to this repository; accordingly you should read through those.

<br>

(I also use this notebook for the documentation of how I set up a local installation of the GeMSE analysis infrastructure and the corresponding `Docker` image. Chapters 3 and 4 are hence irrelevant for your explicit analysis plans but might be helpful for the GeMSE group to future-prove the system.)

<br>

Please keep in mind: I am not a professional programmer and cannot guarantee the code or documentation within the repository of this side project is 100% accurate. So please don't hesitate to contact me in case you have any suggestions, comments or questions (`daniel.baur@physik.uni-freiburg.de`) - I'll do my best to help you out.

<br>
<br>


### Contents of this Repository

- ``Dockerfile``: the `Dockerfile` used to build the `gemse_analysis` image
- ``exemplary_gemse_analysis_notebook.ipynb``: an exemplary notebook to analyze your GeMSE data
- ``exemplary_gemse_measurements``: exemplary GeMSE data that can be analyzed with the ``exemplary_gemse_analysis_notebook.ipynb``
- ``gemseana.py``: some useful scripts imported by ``exemplary_gemse_analysis_notebook.ipynb``
- ``gemse_analysis_docker_compose.yaml``: an exemplary Docker compose file
- ``gemse_analysis_docker_image``: a (.gitignored) directory containing installation data for the Docker image
- ``gemse_analysis_documentation.ipynb``: this file

<br>
<br>

### Table of Contents


0. **[Introduction](#0.-Introduction)**<br>


1. **[Remote Analysis via ssh](#1.-Remote-Analysis-via-ssh)**<br>


2. **[Local Analysis via Docker Image](#2.-Local-Analysis-via-Docker-Image)**<br>


3. **[Local Installation of the Analysis Scripts](#3.-Local-Installation-of-the-Analysis-Scripts)**<br>


4. **[Creating a Docker Image with Executable Analysis Scripts](#4.-Creating-a-Docker-Image-with-Executable-Analysis-Scripts)**<br>

<br>
<br>
<br>
<br>
<br>

# 0. Introduction

<br>
<br>

Hier könnte Ihre Werbung stehen

<br>
<br>
<br>
<br>
<br>

# 1. Remote Analysis via ssh

<br>
<br>

Data recorded with the GeMSE screening facility is/was analyzed utilizing the GeMSE analysis framework installed on ``bcfr102``, a server that was brought from Bern to Freiburg. You can access the machine via ``ssh`` - but only from ``pcfr31``. (In order to do so you first need to generate an ssh key pair. Send the public one to Diego; he needs to register it on the GeMSE machine. Also implement the other one into your ssh configuration file on `pcfr31`. Also see [here](https://askubuntu.com/questions/311558/ssh-permission-denied-publickey) (I had to add `IdentitiesOnly yes`).)

<br>

The following lines give insight to usage of the analysis scripts installed on ``bcfr102``.

<br>

### Data Submission

1. Copy the raw data from your local machine onto `pcfr31`:<br>
`$ scp -r <abspath_to_raw_data_directory> <your_pcfr31_account>@pcfr31.physik.uni-freiburg.de:<abspath_to_intermediate_folder_on_pcfr31>`<br>
e.g.,:<br>
`$ scp -r ./data dbaur@pcfr31.physik.uni-freiburg.de:~/notebooks/ptfesc/data`


2. Log into `pcfr31` (default login and password: ``dbaur`` and ``%b%%%1%%%!``):<br>
`$ ssh -XY <your_pcfr31_account>@pcfr31.physik.uni-freiburg.de`


3. Once logged into `pcfr31`, copy the raw data directory further onto `bcfr102`:
`$ scp -r <abspath_to_intermediate_folder_on_pcfr31> gemse@bcfr102:<abspath_to_raw_data_directory_on_bcfr102>`<br>
e.g.,:<br>
`$ scp -r ~/notebooks/ptfesc/data gemse@bcfr102:~/daniel/data`


4. From `pcfr31` log into `bcfr102`:<br>
`$ ssh -Y gemse@bcfr102`


5. Once the analysis (i.e., the steps below) is finished retrieve the data in the same way that you uploaded it.

<br>

### GeMSE Analysis

The following directories within `bcfr102` are important:
- `~/daniel`: folder Diego created, containing specifically my data
- `~/daniel/20200908_ptfe_bricks_simple_analysis`: folder containing the analysis scripts Diego compiled for me
- `~/daniel/20200904__data`: folder containing the raw data
- `~/daniel/data`: folder containing the raw data (edited: 12th Dec 2020)
- `~/GeMSE_ROOT_scripts`: folder containing the root analysis scripts
- `~/GeMSE_ROOT_scripts_copy_20200904`: backup folder containing the root analysis scripts
- `~/GeMSE_analysis`: folder containing the GeMSE analysis script
- `~/GeMSE_analysis_copy_20200908`: backup folder containing the GeMSE analysis script
The steps below summarize how to run the GeMSE analysis infrastructure.

<br>

1. **Generating a Calibrated Energy Spectrum**<br>
Check out [Thomas Kok's wiki note](https://wiki.uni-freiburg.de/app/doku.php?id=start:gemse:daq:data_workflow).
  - In order to generate a calibrated energy spectrum one first needs to convert the list file saved by the MCA into a .root file. Therefore execute the following:<br><br>
`$ /path/to/backup/scripts/folder/make_rootfile_list /path/to/mca_list_file.txt /path/to/calibration_function`<br><br>
e.g.:<br><br>
`$ ~/GeMSE_ROOT_scripts_copy_20200904/make_rootfile_list ~/daniel/20200904__data/20191010__ptfe_01_gf/2019-10-28_PTFE_Daniel_ch000_Copy20191115_rightAfterClosingGN2valve.txt /home/gemse/measurements/samples/2018-10-08_RockSample_Beda/sample_spectrum/energy_calibration/calibration_spectrum.txt.root_spectrum_0-2050707s.root_calibration_function.root`<br><br>
Hereby the file `/home/gemse/measurements/samples/2018-10-08_RockSample_Beda/sample_spectrum/energy_calibration/calibration_spectrum.txt.root_spectrum_0-2050707s.root_calibration_function.root` provides the necessary energy calibration function. If you do not reference a suchlike file the output root file will be histogrammed in adc channels.<br>

  - Next one needs to generate a spectrum out of the just generated .root file. Execute<br><br>
  `$ /path/to/backup/scripts/folder/make_spectrum_list --file /path/just/generated/root/file.root --energy`<br><br>
  e.g.:<br><br><br>
  `$ ~/GeMSE_ROOT_scripts_copy_20200904/make_spectrum_list --file ~/daniel/20200904__data/20191010__ptfe_01_gf/2019-10-28_PTFE_Daniel_ch000_Copy20191115_rightAfterClosingGN2valve.txt.root --energy`<br><br>
  Hereby the flag `--energy` designates to use the just specified energy calibration. 


2. **Running the actual Analysis**

Now is the time to finally run the analysis:<br><br>
`$ path/to/GeMSE_analysis path/to/config/file.txt`<br><br>
e.g.<br><br>
`$ ~/GeMSE_analysis_copy_20200908/GeMSE_analysis ~/daniel/20200908_ptfe_bricks_simple_analysis/results/kljhsdfkl.txt`

<br>
<br>
<br>
<br>
<br>

# 2. Local Analysis via Docker Image

<br>
<br>

If you have been referred to this repository because you'd like to perform some GeMSE analysis on your local machine then this section is made specifically for you! Instead of the manually executing the GeMSE analysis scripts on a remote server you can set up a GeMSE analysis docker container allowing to locally analyze screening data using automatized scripts in a familiar Jupyter Notebook environment.

<br>

#### Short Introduction to Docker

<br>



<br>

#### Prerequisites: Installing Docker and Pulling the GeMSE Analysis Image

- In order to install `Docker` follow the instructions for your local system in the [official documentation](https://docs.docker.com/engine/install/ubuntu/).


- Obtain the GeMSE analysis Docker image by executing:<br>
`$ sudo docker pull nequiem/gemse_analysis:latest`

<br>

#### Running the Container
Run the `gemseana` container with an interactive terminal (`-it`) based upon the just downloaded `gemse_analysis:1.0` image:<br>
  `$ sudo docker build -t nequiem/gemse_analysis:latest .; sudo docker run -p 8880:8880 --name gemseana -v /home/daniel/Desktop/dtdv:/home/gemse_analysis_infrastructure/ex -it nequiem/gemse_analysis:latest`

<br>

#### Jupyter Notebook

Execute from within the container:<br>
`$ jupyter notebook --port=8880 --no-browser --ip=0.0.0.0 --allow-root`<br>
On your local machine open any browser and access the followiwng URL<br>
`http://127.0.0.1:8880/?token=<token>`<br>
with `<token>` being the forwarded token from the above `jupyter notebook` call.<br>
This now enables you work with a `jupyter notebook` environment with all prerequisites installed.

<br>

#### Notes ####

- **occupied port**: In the example above I used the (arbitrarily chosen) port `8880`. Should that port already be occupied, just choose another one (e.g., `8888`, `8889`, etc...).

- **workflow when modifying the Docker image:**<br>
Within the just opened terminal you can now see whether the system works as inteded. If you'd like to make adjustments to the image you first have to remove the container `gemseana` and also the image `gemse_analysis:1.0`:<br>
  `$ sudo docker rm gemseana; sudo docker rmi gemse_analysis:1.0`<br>
  Once you are happy with your modifications to the image just rebuild the image and run the container:<br>
  `$ sudo docker build -t gemse_analysis:1.0 .; sudo docker run -p 8880:8880 --name gemseana -v /home/daniel/Desktop/dtdv:/home/gemse_analysis_infrastructure/ex -it gemse_analysis:1.0`
<br>
<br>
<br>
<br>
<br>

# 3. Local Installation of the Analysis Scripts

<br>
<br>

This section is meant to document how I set up the GeMSE analysis infrastructure on my private laptop (in order to then maybe develop a docker solution at some point in the future). See also [Thomas Kok's documentation in the wiki](https://wiki.uni-freiburg.de/app/doku.php?id=start:gemse:daq:data_setup). The local installation of the GeMSE analysis infrastructure is located in `/home/daniel/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/`.

<br>

### Installing ROOT

I basically followed the [official instructions](https://root.cern/install/):
- Within the `20210105__local_gemse_analysis_framework` project directory I created the `ROOT` install folder `root`.
- I installed/updated all of the [required dependencies](https://root.cern/install/dependencies/) via: `$ sudo apt-get install <package>`.
- I downloaded and and unpacked the [precompiled `ROOT` binaries for my Ubuntu 18.04 installation](https://root.cern/install/all_releases/) via: `wget https://root.cern/download/root_v6.22.02.Linux-ubuntu18-x86_64-gcc7.5.tar.gz`.
- I edited my `.bashrc` to automatically source the `ROOT` installation by adding the following line:<br>
`source /home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/root/root_v6.22.06.Linux-ubuntu18-x86_64-gcc7.5/root/bin/thisroot.sh`

<br>

### Installing BAT

- Within the `20210105__local_gemse_analysis_framework` project directory I created the `BAT` containing and install folders `bat_0_9_4_1` and `bat_0_9_4_1_install`.
- I downloaded and extracted the [`BATv0.9.4.1` git repository](https://github.com/bat/bat/releases/tag/v0.9.4.1) into the directory `bat_0_9_4_1`.
- From within the `BAT-0.9.4.1` directory I configured the following installation via: `$ ./configure --prefix=/home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/bat_0_9_4_1_install` (by adding the prefix flag you can determine the install directory).
- Also within the `BAT-0.9.4.1` directory I could then run `$ make` and `$ make install`.
- Once the installation is done I made sure to automatically source the `BAT` installation by adding the following lines to my `.bashrc`:<br>
`BATPREFIX="/home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/bat_0_9_4_1_install"`<br>
`export PATH="$BATPREFIX/bin:$PATH"`<br>
`export LD_LIBRARY_PATH="$BATPREFIX/lib:$LD_LIBRARY_PATH"`<br>
`export CPATH="$BATPREFIX/include:$CPATH"`<br>
`export PKG_CONFIG_PATH="$BATPREFIX/lib/pkgconfig:$PKG_CONFIG_PATH"`<br>

<br>

### Compiling Moritz' Repositories: GeMSE_ROOT_scripts and GeMSE_analysis

- Within the `20210105__local_gemse_analysis_framework` project directory I created the containing folders `gemse_root_scripts` and `gemse_analysis`.
- Therein I downloaded and unpacked the git repositories [`GeMSE_ROOT_scripts`](https://github.com/vonsivers/GeMSE_ROOT_scripts) and [`GeMSE_analysis`](https://github.com/vonsivers/GeMSE_analysis) respectively:<br>
`git clone https://github.com/vonsivers/GeMSE_ROOT_scripts.git`<br>
`git clone https://github.com/vonsivers/GeMSE_analysis`<br>
- Unfortunately I had to manually replace `ifstream` with `std::ifstream` within all `.cxx` files for the `GeMSE_ROOT_scripts` repository (probably due to updates of `ROOT` or some `C++` compiler; thank you Sebastian).
- In order to properly compile the executables I executed the following within the respective directories:<br>
`$ make clean`<br>
`$ make`<br>
`$ make install`

<br>

### Comments

Setting up the GeMSE analysis infrastructure (even just partially) on my laptop took more time than I'd like to admit. Below you find some of the thoughts I thought were worth writing down.
- `bcfr102` runs `ROOTv5.34/34`.
- `bcfr102` runs `BATv0.9.4.1`.
- Moritz' `GeMSE_analysis` script does not compile with the most recent (4th January 2021) `BAT` release (!).
- For documentation of Moritz' scripts look at the readme files within his repositories.
- loose ends:
  - energy calibration: I use some already-existing file.
  - energy resolution: I use some already-existing file.
  - efficiency simulation: I use some already-existing file.
  - analysis settings: I use some already-existing settings.

<br>
<br>
<br>
<br>
<br>

# 4. Creating a Docker Image with Executable Analysis Scripts

<br>
<br>

This section is meant to provide documentation (mainly for myself) on how I set up the GeMSE analysis Docker image.

<br>

#### Resources

- [excellent Docker tutorial on YouTube](https://www.youtube.com/watch?v=3c-iBn73dDE)
- contacts: Diego Ramirez, Frank Sauerburger (AG Jakobs), Matthias Gorzellik
- 

<br>

#### Useful Docker Commands

- `$ docker run --rm -it <pathstring_image>`:<br>
  The (probably most important) command `docker run` is used to eventually pull the specified image and run a new container from the specified image.
  - `--rm`:
  - `-it`: running the container interactively will open a terminal within it
  - `-d`: running the container in detached mode will run the containered process in the background
  - `-p:<container_port>:<host_port>`: attache the container to the specified ports
  - `-e<ev_name>=<ev_value>`: specify an environmental variable
  - `--name <custom_container_name>`: specify the container name
  - `--net <network_name>`: specify the container network within which the container is run


- `$ docker images`:<br>
  The command `docker images` displays the locally available images.


- `$ docker ps`:<br>
  The command `docker ps` lists all currently running containers along with relevant information.
  - `-a`: also list stopped containers.


- `$ docker stop <container_id>`:<br>
  The command `docker stop` stops the specified container currently running.


- `$ docker start <container_id>`:<br>
  The command `docker start` starts a stopped container.


- `$ docker exec -it <container_id> /bin/bash`:<br>
  The command `docker exec` is opening up a terminal within the specified container.


- `$ docker rm <container_id>`:<br>
  The command `docker rm` is deleting a container.


- `$ docker rmi <image_id>`:<br>
  The command `docker rmi` is deleting an image.


- `$ docker-compose -f <pathstring_docker_compose_yaml_file> up`:<br>
  The command `docker-compose` will set up the container network specified within the specified `.yaml` file.
  - `-f <pathstring_docker_compse_yaml_file>`: specify the utilized compose file
  - note: depending on whether you specify `up` or `down` the command `docker-compose` will either boot or shut down the containers specified within the indicated compose file.

<br>

#### Installing Docker

In order to install `Docker` I just followed the instruction for my `Ubuntu:18.04` system in the [official documentation](https://docs.docker.com/engine/install/ubuntu/).

<br>

#### Building the Docker Image and Running the Container

- **Setting Up the `Dockerfile`:**<br>
  The so-called `Dockerfile` is basically the blueprint for the image you are about to create. Name it `Dockerfile` is mandatory. The syntax is relatively easy to understand:
  - `FROM`: indicating the image your image is based upon.
  - `SHELL ["/bin/bash", "-c"]`: indicating that the following `RUN` commands are supposed to utilizing the standard Ubuntu shell.
  - `RUN`: Listed here are commands that are executed from WITHIN the image


- **Building the Image:**<br>
  To build the `gemse_analysis:1.0` image execute the following from within the  directory that contains the `Dockerfile` (i.e., `20210114__docker_image_for_gemse_analysis`):<br>
  `$ docker build -t gemse_analysis:1.0 .` (don't forget the `.` indicating the `Dockerfile` saved within this directory)


- **Running the Container:**<br>
  Run the `gemseana` docker container with an interactive terminal (`-it`) based upon the just created `gemse_analysis:1.0` image:<br>
  `$ sudo docker run --name gemseana -it gemse_analysis:1.0`


- **Jupyter Notebook:**<br>
  Execute from within the container:<br>
  `$ jupyter notebook --port=8880 --no-browser --ip=0.0.0.0 --allow-root`<br>
  On your local machine open any browser and access the followiwng URL<br>
  `http://127.0.0.1:8880/?token=<token>`<br>
  with `<token>` being the forwarded token from the above `jupyter notebook` call.<br>
  This now enables you work with a `jupyter notebook` environment with all prerequisites installed.


- **Push the Built Image to a Repository**<br>
  Hier kn
  - Download and install the ``AWS CLI`` following [the official instructions](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux.html#cliv2-linux-install).
  - Modify the credentials for the ``AWS CLI`` following [the official instructions](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html).


- **Notes:**<br>
  - **occupied port**: In the example above I used the (arbitrarily chosen) port `8880`. Should that port already be occupied, just choose another one (e.g., `8888`, `8889`, etc...).
  - **workflow when modifying the Docker image:**<br>
  Within the just opened terminal you can now see whether the system works as inteded. If you'd like to make adjustments to the image you first have to remove the container `gemseana` and also the image `gemse_analysis:1.0`:<br>
  `$ sudo docker rm gemseana; sudo docker rmi gemse_analysis:1.0`<br>
  Once you are happy with your modifications to the image just rebuild the image and run the container:<br>
  `$ sudo docker build -t gemse_analysis:1.0 .; sudo docker run -p 8880:8880 --name gemseana -v /home/daniel/Desktop/dtdv:/home/gemse_analysis_infrastructure/ex -it gemse_analysis:1.0`
  

<br>
<br>
<br>
<br>
<br>


